In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

pd.set_option("display.max_columns", 100)

In [34]:
df = pd.read_csv("../data/raw/Loan_approval_data_2025.csv")

print(df.shape)

df.drop('customer_id',axis=1,inplace=True)
df.head()

(50000, 20)


,age,occupation_status,years_employed,annual_income,credit_score,credit_history_years,savings_assets,current_debt,defaults_on_file,delinquencies_last_2yrs,derogatory_marks,product_type,loan_intent,loan_amount,interest_rate,debt_to_income_ratio,loan_to_income_ratio,payment_to_income_ratio,loan_status
0,40,Employed,17.2,25579,692,5.3,895,10820,0,0,0,Credit Card,Business,600,17.02,0.423,0.023,0.008,1
1,33,Employed,7.3,43087,627,3.5,169,16550,0,1,0,Personal Loan,Home Improvement,53300,14.10,0.384,1.237,0.412,0
2,42,Student,1.1,20840,689,8.4,17,7852,0,0,0,Credit Card,Debt Consolidation,2100,18.33,0.377,0.101,0.034,1
3,53,Student,0.5,29147,692,9.8,1480,11603,0,1,0,Credit Card,Business,2900,18.74,0.398,0.099,0.033,1
4,32,Employed,12.5,63657,630,7.2,209,12424,0,0,0,Personal Loan,Education,99600,13.92,0.195,1.565,0.522,1


In [35]:
numeric_features = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = df.select_dtypes(include=['object']).columns.tolist()

target = "loan_status"

numeric_features = [col for col in numeric_features if col != target]

numeric_features, categorical_features

(['age',
  'years_employed',
  'annual_income',
  'credit_score',
  'credit_history_years',
  'savings_assets',
  'current_debt',
  'defaults_on_file',
  'delinquencies_last_2yrs',
  'derogatory_marks',
  'loan_amount',
  'interest_rate',
  'debt_to_income_ratio',
  'loan_to_income_ratio',
  'payment_to_income_ratio'],
 ['occupation_status', 'product_type', 'loan_intent'])

In [36]:
df[numeric_features] = df[numeric_features].fillna(df[numeric_features].median())
df[categorical_features] = df[categorical_features].fillna(df[categorical_features].mode().iloc[0])

In [37]:
df = pd.get_dummies(df, columns=categorical_features, drop_first=False)

In [38]:
def winsorize_series(s, lower=0.01, upper=0.99):
    return s.clip(
        lower=s.quantile(lower),
        upper=s.quantile(upper)
    )

for col in numeric_features:
    df[col] = winsorize_series(df[col])

In [39]:
df.shape

(50000, 28)

In [40]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

df[numeric_features] = scaler.fit_transform(df[numeric_features])

In [41]:
df.to_csv("../data/processed/loan_data_processed.csv", index=False)

In [42]:
import joblib

joblib.dump(scaler, "../models/scaler.pkl")

['../models/scaler.pkl']